In [1]:
# Import important libraries
import pandas as pd
from datasets import load_dataset 
import transformers
from transformers import AutoProcessor
from transformers import AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import os
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, GPT2TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator
import torch
from torch.utils.data import Dataset, DataLoader
import shutil
import json
from PIL import Image
import datasets
import gc
import numpy as np
import itertools
from tqdm import tqdm
import albumentations as A
import cv2
import shutil
import requests
from matplotlib import pyplot as plt
from torchvision import transforms
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
!pip install tensorflow pydot
!pip install torchcam

2024-05-28 06:04:43.820276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 06:04:43.820385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 06:04:43.935900: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.5 MB/s eta 0:00:0000:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00


In [2]:
train_csv_path = '/kaggle/input/roco-dataset/all_data/train/radiologytraindata.csv'
val_csv_path = '/kaggle/input/roco-dataset/all_data/validation/radiologyvaldata.csv'
test_csv_path = '/kaggle/input/roco-dataset/all_data/test/radiologytestdata.csv'

train_images_path = '/kaggle/input/roco-dataset/all_data/train/radiology/images/'
val_images_path = '/kaggle/input/roco-dataset/all_data/validation/radiology/images/'
test_images_path = '/kaggle/input/roco-dataset/all_data/test/radiology/images/'

In [3]:
df_train = pd.read_csv('/kaggle/input/roco-dataset/all_data/train/radiologytraindata.csv')
df_val = pd.read_csv(val_csv_path, delimiter=',')
df_test = pd.read_csv(test_csv_path, delimiter=',')

In [4]:
def sample_data(df, fraction):
    return df.sample(frac=fraction, random_state=42)

In [5]:
sample_fraction = 0.03

sampled_train_df = sample_data(df_train, sample_fraction)
sampled_val_df = sample_data(df_val, sample_fraction)
sampled_test_df = sample_data(df_test, sample_fraction)

In [6]:
sampled_train_df['images'] = train_images_path + sampled_train_df['name']
sampled_val_df['images'] = val_images_path + sampled_val_df['name']
sampled_test_df['images'] = test_images_path + sampled_test_df['name']

In [7]:
train_folder_path = "/kaggle/working/train"
val_folder_path = "/kaggle/working/val"
test_folder_path = "/kaggle/working/test"

In [8]:
os.makedirs(train_folder_path, exist_ok=True)
os.makedirs(val_folder_path, exist_ok=True)
os.makedirs(test_folder_path, exist_ok=True)

In [9]:
# Function to copy images with checks
def copy_images(sampled_df, destination_folder):
    missing_files = []
    for index, row in sampled_df.iterrows():
        source_file = row["images"]
        file_name = os.path.basename(source_file)
        destination_file = os.path.join(destination_folder, file_name)
        
        if os.path.exists(source_file):
            try:
                shutil.copy(source_file, destination_file)
            except Exception as e:
                print(f"Error copying file {source_file} to {destination_file}: {e}")
        else:
            missing_files.append(source_file)
    
    if missing_files:
        print(f"Total missing files: {len(missing_files)}")
        for file in missing_files:
            print(f"File not found: {file}")

# Copy images for train, val, and test sets
copy_images(sampled_train_df, train_folder_path)
copy_images(sampled_val_df, val_folder_path)
copy_images(sampled_test_df, test_folder_path)

Total missing files: 1
File not found: /kaggle/input/roco-dataset/all_data/validation/radiology/images/PMC5754747_f1000research-6-14507-g0001.jpg


In [10]:
sampled_train_df = sampled_train_df.drop(columns=["images", "id"])
sampled_val_df = sampled_val_df.drop(columns=["images", "id"])
sampled_test_df = sampled_test_df.drop(columns=["images", "id"])

In [11]:
# Convert dataframes to JSON format
train_captions = sampled_train_df.apply(lambda row: {"file_name": row["name"], "text": row["caption"]}, axis=1).tolist()
val_captions = sampled_val_df.apply(lambda row: {"file_name": row["name"], "text": row["caption"]}, axis=1).tolist()
test_captions = sampled_test_df.apply(lambda row: {"file_name": row["name"], "text": row["caption"]}, axis=1).tolist()

In [12]:
# Save data to JSON files
with open(train_folder_path + "/metadata.jsonl", 'w') as f:
    for item in train_captions:
        f.write(json.dumps(item) + "\n")

with open(val_folder_path + "/metadata.jsonl", 'w') as f:
    for item in val_captions:
        f.write(json.dumps(item) + "\n")

with open(test_folder_path + "/metadata.jsonl", 'w') as f:
    for item in test_captions:
        f.write(json.dumps(item) + "\n")

In [13]:
# Load datasets
train_dataset = load_dataset("imagefolder", data_dir=train_folder_path, split="train")
val_dataset = load_dataset("imagefolder", data_dir=val_folder_path, split="train")
test_dataset = load_dataset("imagefolder", data_dir=test_folder_path, split="train")

Resolving data files:   0%|          | 0/1965 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/245 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/246 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
# Define function to convert image to RGB
def convert_to_rgb(image):
    if image.mode != "RGB":
        image = image.convert("RGB")
    return image


# Define the image transformations
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [15]:
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor, tokenizer):
        self.dataset = dataset
        self.processor = processor
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item["image"]
        text = item["text"]

        # Convert image to RGB if necessary
        image = convert_to_rgb(image)

        # Apply preprocessing transformations
        image = preprocess(image)

        # Tokenize the text
        tokenized_text = self.tokenizer(text, padding="max_length", max_length=16, return_tensors="pt", truncation=True)

        # Remove batch dimension
        encoding = {
            "pixel_values": image,
            "input_ids": tokenized_text.input_ids.squeeze(),
            "attention_mask": tokenized_text.attention_mask.squeeze(),
            "labels": tokenized_text.input_ids.squeeze()
        }

        return encoding

In [16]:
# Load the model, processor, and tokenizer
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [18]:
# Create training, validation, and test datasets
train_dataset = ImageCaptioningDataset(train_dataset, feature_extractor, tokenizer)
val_dataset = ImageCaptioningDataset(val_dataset, feature_extractor, tokenizer)
test_dataset = ImageCaptioningDataset(test_dataset, feature_extractor, tokenizer)

In [19]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=default_data_collator)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=default_data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=default_data_collator)

In [20]:
from transformers.trainer_callback import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    num_train_epochs=20,
    save_total_limit=3,
    save_strategy="epoch",
    load_best_model_at_end=True,  # Ensuring the best model is loaded at the end
    metric_for_best_model="eval_loss",  # Define the metric for the best model
    greater_is_better=False  # Specify whether a higher metric is better (for eval_loss, lower is better)
)

In [21]:
!pip install rouge_score

rouge = datasets.load_metric("rouge")

def get_embeddings(text_list, max_length=20):
    tokenized_output = tokenizer(text_list, padding="max_length", max_length=max_length, return_tensors="pt", truncation=True)
    return tokenized_output.input_ids

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute Rouge2
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    # Set max_length for tokenization
    max_length = 20
    
    # Compute Cosine Similarity
    pred_embeddings = get_embeddings(pred_str, max_length=max_length).numpy()
    label_embeddings = get_embeddings(label_str, max_length=max_length).numpy()

    cosine_sim = cosine_similarity(pred_embeddings, label_embeddings).diagonal().mean()

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
        "cosine_similarity": round(cosine_sim, 4),
    }

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e27863ec6cc408a888840003946a94a2c0446ef3518747b55a851f7379d91d9b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


/tmp/ipykernel_34/303884833.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
# Instantiate the trainer
trainer = Seq2SeqTrainer(
    tokenizer=tokenizer,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
    callbacks=[early_stopping]

)

# Start training
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Cosine Similarity
1,3.861300,3.626656,0.046200,0.056400,0.048700,0.583800
2,3.074600,3.550105,0.043900,0.048400,0.043900,0.608400
3,3.069800,3.570822,0.039300,0.054900,0.044700,0.545800
4,2.530800,3.681729,0.045800,0.063900,0.052300,0.563300
5,2.226700,3.860579,0.044900,0.058100,0.049400,0.565500


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Removed shared tensor {'decoder.lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn

TrainOutput(global_step=1230, training_loss=2.9639605514402314, metrics={'train_runtime': 818.7233, 'train_samples_per_second': 47.977, 'train_steps_per_second': 6.009, 'total_flos': 1.772155410799657e+18, 'train_loss': 2.9639605514402314, 'epoch': 5.0})

In [23]:
# Save the model, tokenizer, and processor
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
feature_extractor.save_pretrained("./fine_tuned_model")

['./fine_tuned_model/preprocessor_config.json']

In [24]:
# Load the fine-tuned model, tokenizer, and processor
fine_tuned_model = VisionEncoderDecoderModel.from_pretrained("./fine_tuned_model")
fine_tuned_tokenizer = GPT2TokenizerFast.from_pretrained("./fine_tuned_model")
fine_tuned_processor = ViTImageProcessor.from_pretrained("./fine_tuned_model")

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [25]:
# Define the function to generate captions for the test dataset
def generate_captions_with_ground_truth(model, tokenizer, processor, dataloader, device):
    model.eval()
    generated_captions = []
    real_captions = []
    
    for batch in dataloader:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        with torch.no_grad():
            output_ids = model.generate(pixel_values, max_length=16, num_beams=4)
        
        decoded_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        generated_captions.extend(decoded_preds)
        
        # Convert label IDs to text
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        real_captions.extend(decoded_labels)
    
    return generated_captions, real_captions

# Generate captions for the test dataset
generated_captions, real_captions = generate_captions_with_ground_truth(model, tokenizer, feature_extractor, test_dataloader, device)

# Print the generated and real captions
for idx, (generated_caption, real_caption) in enumerate(zip(generated_captions, real_captions)):
    print(f"Image {idx+1}:")
    print(f"    Real Caption: {real_caption}")
    print(f"    Generated Caption: {generated_caption}")
    print()

Image 1:
    Real Caption:  Pancreatic lesion of ultrasonic finding. Abdominal ultrasound revealed an
    Generated Caption:  Transesophageal echocardiography (TEE) of

Image 2:
    Real Caption:  Computed tomography: well delimited pulmonary nodule of 13 mm in diameter
    Generated Caption:  Computed tomography (CT) scan of the abdomen and pelvis showing

Image 3:
    Real Caption:  CT guided core biopsy of infracolic omental cake.

    Generated Caption:  Computed tomography scan of the abdomen and pelvis.


Image 4:
    Real Caption:  Abdominopelvic contrast-enhanced computed tomography. Multiple liver
    Generated Caption:  Computed tomography (CT) scan of the abdomen and pelvis showing

Image 5:
    Real Caption:  Left arm venography in a 1-year-old female with a malfunctioning
    Generated Caption:   Anterior-posterior T2-weighted image of the

Image 6:
    Real Caption:  Focal edema, no total knee joint replacement, in a 58-year
    Generated Caption:  Axial T2-weighted magne

In [26]:
generated_captions

[' Transesophageal echocardiography (TEE) of',
 ' Computed tomography (CT) scan of the abdomen and pelvis showing',
 ' Computed tomography scan of the abdomen and pelvis.\n',
 ' Computed tomography (CT) scan of the abdomen and pelvis showing',
 '  Anterior-posterior T2-weighted image of the',
 ' Axial T2-weighted magnetic resonance image of the brain showing a',
 ' X-ray of the pelvis showing the fracture.\n',
 ' Computed tomography (CT) scan of the abdomen and pelvis showing',
 ' Axial T2-weighted magnetic resonance image of the brain showing a',
 ' X-ray of the pelvis showing the fracture.\n',
 ' Computed tomography scan of the abdomen and pelvis.\n',
 ' Computed tomography (CT) scan of the abdomen and pelvis showing',
 ' X-ray of the pelvis showing the fracture.\n',
 ' Transesophageal echocardiogram.\n',
 ' Computed tomography scan of the abdomen and pelvis.\n',
 ' Transesophageal echocardiography (TEE) of',
 ' Panoramic radiograph of the pelvis.\n',
 ' Chest radiograph of the patie

In [27]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the tokenizer and model
model_name = 'facebook/m2m100_418M'
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Function to translate text
def translate(texts, src_lang, tgt_lang, tokenizer, model):
    tokenizer.src_lang = src_lang
    encoded_texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded_texts, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    translations = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in generated_tokens]
    return translations

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [28]:
translations = translate(generated_captions, "en", "ar", tokenizer, model)
for text, translation in zip(generated_captions, translations):
    print(f"Input: {text.strip()}")
    print(f"Translation: {translation.strip()}\n")

Input: Transesophageal echocardiography (TEE) of
Translation: التهاب القولون (TEE) من

Input: Computed tomography (CT) scan of the abdomen and pelvis showing
Translation: توموغرافيا الكمبيوتر (CT) فحص البطن والبطن يظهر

Input: Computed tomography scan of the abdomen and pelvis.
Translation: فحص التوموغرافيا الحاسوبية للبطن والبطن.

Input: Computed tomography (CT) scan of the abdomen and pelvis showing
Translation: توموغرافيا الكمبيوتر (CT) فحص البطن والبطن يظهر

Input: Anterior-posterior T2-weighted image of the
Translation: الصورة الخلفية T2 المرتفعة

Input: Axial T2-weighted magnetic resonance image of the brain showing a
Translation: الصورة المغناطيسية T2 المهمة في الدماغ التي تظهر

Input: X-ray of the pelvis showing the fracture.
Translation: الأشعة السينية من الحبل الشوكي التي تظهر الانكماش.

Input: Computed tomography (CT) scan of the abdomen and pelvis showing
Translation: توموغرافيا الكمبيوتر (CT) فحص البطن والبطن يظهر

Input: Axial T2-weighted magnetic resonance image of the b